# Weak Mixed k-Metric Dimensions
Za začetek napišimo CLP program, ki bo vrnil $wmdim_k(G)$ in pripadajoč "resolving set" $S$. Pred tem pa definirajmo še razdaljo med vozlišči in povezavami.

In [6]:
from sage.numerical.mip import MixedIntegerLinearProgram

def razdalja(moznost, vozlisce, G):
    c, a = moznost
    if c == 'e':
        U, V = a
        return min(G.distance(U, vozlisce), G.distance(V, vozlisce))
    else:
        return G.distance(a, vozlisce)

In [7]:
def CLP_weak_k_dim(G, k):
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True)

    V = G.vertices()
    E = G.edges(labels=False)

    moznosti = [('v', v) for v in V] + [('e', e) for e in E]

    p.set_objective(sum(x[v] for v in V))
    
    for a, b in Combinations(moznosti, 2):
        p.add_constraint(
            sum(abs(razdalja(a, v, G) - razdalja(b, v, G)) * x[v] for v in V) >= k
            )

    wmdim_k = p.solve()
    mnozica_S = [v for v in V if p.get_values(x[v]) > 0.5]

    return (wmdim_k, mnozica_S)

Sedaj definirajmo še funkcijo $\kappa''(G)$, ki je po definiciji največja vrednost $k$ za katero je $wmdim_k(G)$ še definirana.

In [8]:
def kappa_2_crti(G):
    k = 1
    while True:
        try:
            CLP_weak_k_dim(G, k)
            k += 1
        except:
            return k - 1